In [ ]:
#!pip install pandas spacy swifter
import pandas as pd
import spacy
import swifter
import ast
import json
import numpy as np

In [7]:
import json
data = pd.read_json("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Model_data/validation_set.json")

data.to_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Model_data/validation_set.csv", index=False)



In [8]:


hello = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Model_data/validation_set.csv")

hello.head()

,paragraph,sentence_nr,text,speaker,party,preceding_sentence,succeeding_sent,current_speaker_in_government,parties_in_government,date,label
0,12653,3,Fru Margrete Auken benyttede nogle forenklende...,Arne Melchior,CD,For at få dét til at rime sammen skal man vis...,"Jeg ved ikke, hvor fru Margrete Auken har dét ...",False,"['S', 'RV']",1998-06-16,1
1,14682,1,"Og længe leve fordommene, specielt fra fru Eli...",Brian Mikkelsen,KF,"Man skal nogle gange høre meget herinde, før ø...","Jeg kunne godt tænke mig at vide, om hun har l...",False,"['S', 'RV']",1998-10-22,1
2,27558,24,"Jeg tror, jeg har ret, når jeg siger, at de m...",Poul Nyrup Rasmussen,S,"Det andet, jeg vil nævne, er det sociale arbej...","Det gælder narkomaner, det gælder en række an...",True,"['S', 'RV']",1999-05-12,0
3,28378,33,"Det er ganske smart lavet af venstrefløjen, s...",Birthe Rønn Hornbech,V,Det er nogle partsindlæg.,Det hele bygger på et pres fra venstrefløjen.,False,"['S', 'RV']",1999-05-25,1
4,3439,0,"Jeg skal blot beklage, at hr. Poul Erik Dyrlun...",Charlotte Antonsen,V,NaN,"Når man taler om omfordeling, er der altså tal...",False,"['S', 'RV']",1997-12-02,1


In [3]:
#python -m spacy download da_core_news_sm
nlp = spacy.load("da_core_news_sm")

def split_paragraph(paragraph: str):
    doc = nlp.pipe([paragraph], batch_size=1, n_process=1)
    return [sent.text.strip() for sent in list(doc)[0].sents]


def danish_sentences_extraction(data, text_column):

    sentences = {}

    for para in data.index:
        text_sentences = data.loc[para][text_column]
        
        blame_sentence_dict = {}
        for indx in range(len(text_sentences)):
            blame_sentence = text_sentences[indx]
            blame_sentence_dict[indx] = blame_sentence
        
        
        sentences[para] = blame_sentence_dict

    return sentences


#------------------------------------------#
def convert_to_json_and_write(file_name, sentences):
    with open(f'/work/MarkusLundsfrydJensen#1865/inferece_data/{file_name}.json', 'w') as file:
        file.write(json.dumps(sentences, indent=4))



    #preprocess data for label studio
    # Load your data
    with open(f'/work/MarkusLundsfrydJensen#1865/inferece_data/{file_name}.json', "r", encoding="utf-8") as f:
        data = json.load(f)

    flattened = []

    for paragraph, sentences in data.items():
        for sentence_nr, text in sentences.items():
            flattened.append({
                "paragraph": paragraph,
                "sentence_nr": sentence_nr,
                "text": text
            })

    # Save in a format Label Studio can import
    with open(f'/work/MarkusLundsfrydJensen#1865/inferece_data/{file_name}.json', "w", encoding="utf-8") as f:
        json.dump(flattened, f, ensure_ascii=False, indent=2)

    return


#final data is csv file as output of PolDebate model

def json_append_meta_data(file_name, data):
    meta_data = data[['Unnamed: 0','speaker','party']]
    meta_data = meta_data.replace({np.nan: None})
    #meta_data.head()


    #connect label studio data with meta data


    # Load the flattened sentence JSON
    with open(f"/work/MarkusLundsfrydJensen#1865/inferece_data/{file_name}.json", "r", encoding="utf-8") as f:
        sentences = json.load(f)

    # Load metadata
    meta = meta_data

    # Convert metadata to dict for fast lookup
    meta_dict = meta.set_index("Unnamed: 0").to_dict(orient="index")

    # Merge
    for item in sentences:
        paragraph = int(item["paragraph"])
        if paragraph in meta_dict:

            item.update(meta_dict[paragraph])

    # Save merged dataset
    with open(f'/work/MarkusLundsfrydJensen#1865/inferece_data/{file_name}.json', "w", encoding="utf-8") as f:
        json.dump(sentences, f, ensure_ascii=False, indent=2)


    return




#find parties in government by dataset
def find_government(gov_data, date):
    match = gov_data[(gov_data["Start Date"] <= date) & (gov_data["End Date"] >= date)]
    if not match.empty:

        parties = match['Party Letter']
        parties = ast.literal_eval(parties.iloc[0])
        
        return parties
    else:
        print('empty')
        return None


#append context and government related data to the json file
def json_government_and_context(file_name, data, government_data):

    # Load the flattened sentence JSON
    with open(f"/work/MarkusLundsfrydJensen#1865/inferece_data/{file_name}.json", "r", encoding="utf-8") as f:
        json_data = json.load(f)

    for entry in json_data:
        paragraph_nr = int(entry['paragraph'])
        sentence_nr = int(entry['sentence_nr'])

        #initialize information on current paragraph
        temp_data = data.loc[paragraph_nr]

        #Find out if speaker is member of party in Government
        party = entry['party']
        date = temp_data['date']
        parties_gov = find_government(government_data, date)
        if party in parties_gov:
            in_gov = True
        else:
            in_gov = False
        
        #make into dict

        context_dict = {'current_speaker_in_government': in_gov,
                        'parties_in_government': parties_gov,
                        'date': str(date)
                        }

        #update
        entry.update(context_dict)

    #save data
    with open(f"/work/MarkusLundsfrydJensen#1865/inferece_data/{file_name}.json", "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)
    return


import json

def preprocess_json(input_path, output_path=None):
    """
    Preprocesses a JSON file by filtering out entries based on the 'text' key.
    
    Criteria for deletion:
      - 'text' is missing or empty
      - 'text' length is <= 3
      - 'text' contains '(' or ')'
    
    Parameters:
        input_path (str): Path to the input JSON file.
        output_path (str, optional): If provided, saves the filtered JSON here.
    
    Returns:
        list: The filtered list of JSON entries.
    """
    # Load JSON file
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Filter entries
    filtered_data = [
        entry for entry in data
        if 'text' in entry
        and entry['text']
        and len(entry['text']) > 3
        and '(' not in entry['text']
        and ')' not in entry['text']
    ]

    # Optionally save to a new file
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(filtered_data, f, ensure_ascii=False, indent=4)

    return filtered_data

    


In [3]:
#Fist create inference dataset

df = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Corp_Folketing_V2.csv")

# Convert the column to datetime (specify format if necessary)
df["date"] = pd.to_datetime(df["date"], errors='coerce', dayfirst=True)

# Filter rows after January 1, 2000
filtered_df = df[df["date"] > "2000-01-01"]

#filtered_df.pop("Unnamed: 0")

filtered_df.head()

,Unnamed: 0,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
36314,36314,2000-11-01,1) Spørgsmål til ministrene.,2,Henning Grove,NaN,NaN,True,181,Til at besvare spørgsmål i spørgetimen i dag h...,DK-Folketing,DNK
36315,36315,2000-11-01,1) Spørgsmål til ministrene.,3,Frank Dahlgaard,UP,NaN,False,187,Ministeren er jo ikke bare minister for fødeva...,DK-Folketing,DNK
36316,36316,2000-11-01,1) Spørgsmål til ministrene.,4,Henrik Dam Kristensen,S,379.0,False,175,Først vil jeg gerne sige til hr. Frank Dahlgaa...,DK-Folketing,DNK
36317,36317,2000-11-01,1) Spørgsmål til ministrene.,5,Frank Dahlgaard,UP,NaN,False,173,"Ja, vi har en stor eksport, men ministeren er ...",DK-Folketing,DNK
36318,36318,2000-11-01,1) Spørgsmål til ministrene.,6,Henrik Dam Kristensen,S,379.0,False,138,"Hr. Frank Dahlgaard har jo fuldstændig ret i, ...",DK-Folketing,DNK


In [ ]:
import ipywidgets
filtered_df['segmented_text'] = filtered_df['text'].swifter.apply(split_paragraph)



Pandas Apply:   0%|          | 0/293615 [00:00<?, ?it/s]

/tmp/ipykernel_764/2690031985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['segmented_text'] = filtered_df['text'].swifter.apply(split_paragraph)


In [ ]:
#extract dish sentences and make into json 
# format using modified function and inspiration from prelim blame detection

In [ ]:
all_sentences = danish_sentences_extraction(filtered_df, text_column = 'segmented_text')

#all_sentences

20
6
6
6
4
3


TypeError: object of type 'float' has no len()

In [ ]:

convert_to_json_and_write(file_name = 'inference_data',sentences = all_sentences)

In [ ]:

json_append_meta_data(file_name = 'inference_data', data = filtered_df)

In [4]:
#get government data
regerings_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/danish_govs.csv")

#make date related columns into datetime objects
regerings_data["Start Date"] = pd.to_datetime(regerings_data["Start Date"], format="%Y-%m-%d")
regerings_data["End Date"]   = pd.to_datetime(regerings_data["End Date"], format="%Y-%m-%d")

In [5]:
json_government_and_context(file_name = 'inference_data', data = filtered_df, government_data = regerings_data)


In [6]:
#DO some cleanup

preprocess_json(input_path = "/work/MarkusLundsfrydJensen#1865/inferece_data/inference_data.json", output_path="/work/MarkusLundsfrydJensen#1865/inferece_data/preprocessed_inference_data.json")



[{'paragraph': '36314',
  'sentence_nr': '0',
  'text': 'Til at besvare spørgsmål i spørgetimen i dag har  statsministeren udpeget fødevareministeren.',
  'speaker': 'Henning Grove',
  'party': None,
  'current_speaker_in_government': False,
  'parties_in_government': ['S', 'RV'],
  'date': '2000-11-01 00:00:00'},
 {'paragraph': '36314',
  'sentence_nr': '1',
  'text': 'Der er indtil dette øjeblik tilmeldt fem hovedspørgere  til fødevareministeren til denne spørgetime.',
  'speaker': 'Henning Grove',
  'party': None,
  'current_speaker_in_government': False,
  'parties_in_government': ['S', 'RV'],
  'date': '2000-11-01 00:00:00'},
 {'paragraph': '36314',
  'sentence_nr': '2',
  'text': 'Er der flere,  der ønsker at tilmelde sig som hovedspørgere?',
  'speaker': 'Henning Grove',
  'party': None,
  'current_speaker_in_government': False,
  'parties_in_government': ['S', 'RV'],
  'date': '2000-11-01 00:00:00'},
 {'paragraph': '36314',
  'sentence_nr': '4',
  'text': 'Det  er ikke tilfælde

In [ ]:
# read in newer data
#fix date column
#find where to concat
#fix paragraph nr
#sentence segmentation
#reformat into json
#preprocess
#merge

In [4]:
import pandas as pd

df = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Corp_Folketing_V2.csv")
new_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/speeches.csv")


In [5]:
new_data.pop("period")
new_data.pop("Unnamed: 0")
new_data.pop("procedure_ID")
new_data.pop("link")


0         https://www.ft.dk/forhandlinger/20072/20072M08...
1         https://www.ft.dk/forhandlinger/20072/20072M08...
2         https://www.ft.dk/forhandlinger/20072/20072M08...
3         https://www.ft.dk/forhandlinger/20072/20072M08...
4         https://www.ft.dk/forhandlinger/20072/20072M08...
                                ...                        
716802    https://www.ft.dk/forhandlinger/20221/20221M00...
716803    https://www.ft.dk/forhandlinger/20221/20221M00...
716804    https://www.ft.dk/forhandlinger/20221/20221M00...
716805    https://www.ft.dk/forhandlinger/20221/20221M00...
716806    https://www.ft.dk/forhandlinger/20221/20221M00...
Name: link, Length: 716807, dtype: object

In [23]:
# Convert the column to datetime (specify format if necessary)
new_data['date'] = pd.to_datetime(new_data['date'], format='%Y-%m-%d')
new_data.head()

,speaker,text,party,date,agenda,speechnumber,partyfacts_ID,chair
772180,Formanden,Sophie Hæstorp Andersen (S) har søgt om orlov ...,-,2008-09-09,Spørgsmål om meddelelse af orlov til og indkal...,1,NaN,True
772181,Formanden,Fra Udvalget til Valgs Prøvelse har jeg modtag...,-,2008-09-09,Indstilling fra Udvalget til Valgs Prøvelse: G...,2,NaN,True
772182,Formanden,Der stemmes om udvalgets indstilling. Der kan ...,-,2008-09-09,Indstilling fra Udvalget til Valgs Prøvelse: G...,3,NaN,True
772183,Formanden,Der er ikke stillet ændringsforslag. Er der no...,-,2008-09-09,3. behandling af lovforslag nr. L 187: Forslag...,4,NaN,True
772184,Formanden,Der stemmes om lovforslagets endelige vedtagel...,-,2008-09-09,3. behandling af lovforslag nr. L 187: Forslag...,5,NaN,True


716807

In [ ]:
'''#Find date of latest speech in original data






latest_datapoint

df[(df["date"] == "2018-12-12")].head()

new_data[(new_data["date"] == "2018-12-12")].head(-6)
'''

In [26]:
print(new_data['date'].dtype)


datetime64[ns]


In [27]:
#fix paragraph_number
#get last paragraph number of old data
print(len(new_data))
latest_date = max(df["date"])
hello = df[(df["date"] == latest_date)]

latest_speechnumber = max(hello["speechnumber"])

latest_datapoint = df[(df["date"] == latest_date) & (df["speechnumber"] == latest_speechnumber)]
latest_datapoint = latest_datapoint.index[0] +1

new_data = new_data[new_data["date"] > latest_date
]
#df = df.reset_index(drop=True)
new_data.index = range(latest_datapoint, latest_datapoint + len(new_data))
print(len(new_data))
new_data.head()



716807
82738


,speaker,text,party,date,agenda,speechnumber,partyfacts_ID,chair
772180,Formanden (Pia Kjærsgaard),"Forhandlingen er sluttet, og vi går til afstem...",-,2019-07-05,Fortsættelse af forespørgsel nr. F 42 [afstemn...,1,NaN,True
772181,Formanden (Pia Kjærsgaard),Der er ikke stillet ændringsforslag. Er der no...,-,2019-07-05,3. behandling af lovforslag nr. L 191: Forslag...,2,NaN,True
772182,Formanden (Pia Kjærsgaard),Der stemmes om lovforslagets endelige vedtagel...,-,2019-07-05,3. behandling af lovforslag nr. L 191: Forslag...,3,NaN,True
772183,Formanden (Pia Kjærsgaard),Der er ikke stillet ændringsforslag. Er der no...,-,2019-07-05,3. behandling af lovforslag nr. L 222: Forslag...,4,NaN,True
772184,Skatteministeren (Karsten Lauritzen),"Jeg skal bare ganske kort sige, at der har vær...",V,2019-07-05,3. behandling af lovforslag nr. L 222: Forslag...,5,1204.0,False


In [30]:
#now do sentence_segmentation

import ipywidgets
new_data['segmented_text'] = new_data['text'].swifter.apply(split_paragraph)

Pandas Apply:   0%|          | 0/82738 [00:00<?, ?it/s]

/tmp/ipykernel_8952/2620428599.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['segmented_text'] = new_data['text'].swifter.apply(split_paragraph)


In [31]:
new_data.head()

,speaker,text,party,date,agenda,speechnumber,partyfacts_ID,chair,segmented_text
772180,Formanden (Pia Kjærsgaard),"Forhandlingen er sluttet, og vi går til afstem...",-,2019-07-05,Fortsættelse af forespørgsel nr. F 42 [afstemn...,1,NaN,True,"[Forhandlingen er sluttet, og vi går til afste..."
772181,Formanden (Pia Kjærsgaard),Der er ikke stillet ændringsforslag. Er der no...,-,2019-07-05,3. behandling af lovforslag nr. L 191: Forslag...,2,NaN,True,"[Der er ikke stillet ændringsforslag., Er der ..."
772182,Formanden (Pia Kjærsgaard),Der stemmes om lovforslagets endelige vedtagel...,-,2019-07-05,3. behandling af lovforslag nr. L 191: Forslag...,3,NaN,True,[Der stemmes om lovforslagets endelige vedtage...
772183,Formanden (Pia Kjærsgaard),Der er ikke stillet ændringsforslag. Er der no...,-,2019-07-05,3. behandling af lovforslag nr. L 222: Forslag...,4,NaN,True,"[Der er ikke stillet ændringsforslag., Er der ..."
772184,Skatteministeren (Karsten Lauritzen),"Jeg skal bare ganske kort sige, at der har vær...",V,2019-07-05,3. behandling af lovforslag nr. L 222: Forslag...,5,1204.0,False,"[Jeg skal bare ganske kort sige, at der har væ..."


In [32]:
newer_sentences = danish_sentences_extraction(new_data, text_column = 'segmented_text')

In [33]:
convert_to_json_and_write(file_name = 'newer_inference_data',sentences = newer_sentences)

In [35]:
new_data["Unnamed: 0"] = new_data.index
new_data.head()

/tmp/ipykernel_8952/1280509453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Unnamed: 0"] = new_data.index


,speaker,text,party,date,agenda,speechnumber,partyfacts_ID,chair,segmented_text,Unnamed: 0
772180,Formanden (Pia Kjærsgaard),"Forhandlingen er sluttet, og vi går til afstem...",-,2019-07-05,Fortsættelse af forespørgsel nr. F 42 [afstemn...,1,NaN,True,"[Forhandlingen er sluttet, og vi går til afste...",772180
772181,Formanden (Pia Kjærsgaard),Der er ikke stillet ændringsforslag. Er der no...,-,2019-07-05,3. behandling af lovforslag nr. L 191: Forslag...,2,NaN,True,"[Der er ikke stillet ændringsforslag., Er der ...",772181
772182,Formanden (Pia Kjærsgaard),Der stemmes om lovforslagets endelige vedtagel...,-,2019-07-05,3. behandling af lovforslag nr. L 191: Forslag...,3,NaN,True,[Der stemmes om lovforslagets endelige vedtage...,772182
772183,Formanden (Pia Kjærsgaard),Der er ikke stillet ændringsforslag. Er der no...,-,2019-07-05,3. behandling af lovforslag nr. L 222: Forslag...,4,NaN,True,"[Der er ikke stillet ændringsforslag., Er der ...",772183
772184,Skatteministeren (Karsten Lauritzen),"Jeg skal bare ganske kort sige, at der har vær...",V,2019-07-05,3. behandling af lovforslag nr. L 222: Forslag...,5,1204.0,False,"[Jeg skal bare ganske kort sige, at der har væ...",772184


In [36]:
json_append_meta_data(file_name = 'newer_inference_data', data = new_data)

In [37]:
#get government data
regerings_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/danish_govs.csv")

#make date related columns into datetime objects
regerings_data["Start Date"] = pd.to_datetime(regerings_data["Start Date"], format="%Y-%m-%d")
regerings_data["End Date"]   = pd.to_datetime(regerings_data["End Date"], format="%Y-%m-%d")

In [38]:
json_government_and_context(file_name = 'newer_inference_data', data = new_data, government_data = regerings_data)


In [39]:
#DO some cleanup

preprocess_json(input_path = "/work/MarkusLundsfrydJensen#1865/inferece_data/newer_inference_data.json", output_path="/work/MarkusLundsfrydJensen#1865/inferece_data/preprocessed_newer_inference_data.json")



[{'paragraph': '772180',
  'sentence_nr': '0',
  'text': 'Forhandlingen er sluttet, og vi går til afstemning om de fremsatte forslag til vedtagelse.',
  'speaker': 'Formanden (Pia Kjærsgaard)',
  'party': '-',
  'current_speaker_in_government': False,
  'parties_in_government': ['S'],
  'date': '2019-07-05 00:00:00'},
 {'paragraph': '772180',
  'sentence_nr': '1',
  'text': 'Der foreligger to forslag.',
  'speaker': 'Formanden (Pia Kjærsgaard)',
  'party': '-',
  'current_speaker_in_government': False,
  'parties_in_government': ['S'],
  'date': '2019-07-05 00:00:00'},
 {'paragraph': '772180',
  'sentence_nr': '3',
  'text': 'Afstemningen er slut.',
  'speaker': 'Formanden (Pia Kjærsgaard)',
  'party': '-',
  'current_speaker_in_government': False,
  'parties_in_government': ['S'],
  'date': '2019-07-05 00:00:00'},
 {'paragraph': '772180',
  'sentence_nr': '5',
  'text': 'Forslag nr. V 92 er vedtaget.',
  'speaker': 'Formanden (Pia Kjærsgaard)',
  'party': '-',
  'current_speaker_in_go

In [ ]:
#merge data

#OBS inspect data befor running the beneith chunk


In [40]:
def merge_json_files(json_files, output_file_path):
    merged_data = []

    for file in json_files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            merged_data.extend(data)  # add the list from this file to the merged_data

    # Save the merged list to a new JSON file
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, ensure_ascii=False, indent=2)

    return

In [41]:
json_files = ["/work/MarkusLundsfrydJensen#1865/inferece_data/preprocessed_inference_data.json", "/work/MarkusLundsfrydJensen#1865/inferece_data/preprocessed_newer_inference_data.json"]

merge_json_files(json_files, output_file_path = "/work/MarkusLundsfrydJensen#1865/inferece_data/final_inference_data.json" )